# **Чистка и предобработка данных**

In [234]:
import pandas as pd  #импорт библиотек
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

In [235]:
# загрузка файла
!gdown --id 19nyWryasWInDpH_4la_wA4E11LmF_U0R # загрузка файла

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=19nyWryasWInDpH_4la_wA4E11LmF_U0R
To: /content/final_task.csv
100% 3.97M/3.97M [00:00<00:00, 194MB/s]


In [236]:
# посмотрим, что представляет из себя датасет
dataframe = pd.read_csv('final_task.csv')
dataframe.head(5)

,product_id,title,price,sales,feedbacks,seller,seller_rating,Процессор,Оперативная память,Жесткий диск,Видеопроцессор,Операционная система,Гарантийный срок,Страна производства,Габариты товара,Габариты товара (с упаковкой)
0,10148385,Системный блок OFFICE 120 WB 0710623 J1800/4Gb...,10805₽,Купили более 400 раз,7 отзывов,NaN,NaN,"{'Процессор_тип': 'Intel Celeron', 'Количество...","{'Тип оперативной памяти': 'DDR 3', 'Объем опе...","{'Объем накопителя HDD': None, 'Объем накопите...",Intel HD Graphics,отсутствует,3года,Россия,"{'Ширина предмета': '28.5 см', 'Глубина предме...","{'Длина упаковки': '43.5 см', 'Ширина упаковки..."
1,17877962,Системный блок,32900₽,NaN,0 отзывов,NaN,NaN,"{'Процессор_тип': 'Intel Core i5', 'Количество...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай,"{'Ширина предмета': None, 'Глубина предмета': ...","{'Длина упаковки': '43 см', 'Ширина упаковки':..."
2,17880420,Компьютер Hp Prodesk 400 g6 mt,35720₽,NaN,0 отзывов,NaN,NaN,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,NaN,"{'Ширина предмета': None, 'Глубина предмета': ...","{'Длина упаковки': None, 'Ширина упаковки': No..."
3,19347937,Офисный Компьютер Robotcomp Казначей V1,39237₽,Купили более 10 раз,1 отзыв,Robotcomp,4.7,"{'Процессор_тип': 'Intel Core i5', 'Количество...","{'Тип оперативной памяти': 'DDR 4', 'Объем опе...","{'Объем накопителя HDD': None, 'Объем накопите...",Intel UHD Graphics 630,windows пробная,3 Года (36 месяцев),Россия,"{'Ширина предмета': '28 см', 'Глубина предмета...","{'Длина упаковки': '60 см', 'Ширина упаковки':..."
4,19348951,Игровой Компьютер Robotcomp М16 2.0 V1,76188₽,Купили более 200 раз,94 отзыва,Robotcomp,4.7,"{'Процессор_тип': 'Intel Core i5', 'Количество...","{'Тип оперативной памяти': 'DDR 4', 'Объем опе...","{'Объем накопителя HDD': None, 'Объем накопите...",NVIDIA GeForce GTX 1660,windows пробная,3 Года (36 месяцев),Россия,"{'Ширина предмета': '20 см', 'Глубина предмета...","{'Длина упаковки': '50 см', 'Ширина упаковки':..."


In [237]:
# смотрим, какие типы данных содержит датасет, есть ли пропущенные значения
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4500 entries, 0 to 4499
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     4500 non-null   int64  
 1   title                          4500 non-null   object 
 2   price                          4499 non-null   object 
 3   sales                          1164 non-null   object 
 4   feedbacks                      4500 non-null   object 
 5   seller                         4391 non-null   object 
 6   seller_rating                  4389 non-null   float64
 7   Процессор                      4500 non-null   object 
 8   Оперативная память             4500 non-null   object 
 9   Жесткий диск                   4500 non-null   object 
 10  Видеопроцессор                 4500 non-null   object 
 11  Операционная система           4500 non-null   object 
 12  Гарантийный срок               2648 non-null   o

**Предварительный план действий по предобработке:**
*   нам нужны для анализа персональные компьютеры, за исключением мини-ПК и моноблоков, проверить на их наличие;
*   изменить названия колонок так, чтобы они были на одном языке для сохранения единого стиля датасета;
*   price - изменить формат на числовой, убрать символ рубля;
*   sales - перевести в ранги;
*   feedback - очистить, перевести в числовой формат;
*   seller, Габариты товара, Габариты товара (с упаковкой)  - для цели анализа не будут использованы, удалю;
*   seller_rating - формат удобен для анализа, оставляю;
*   Процессор, Жесткий диск, Оперативная память, Операционная система, Видеопроцессор - достать данные из словаря;
*   Гарантийный срок - перевести в формат даты.


In [238]:
# создадим копию датасета
df = dataframe.copy()

In [239]:
# меняем названия колонок с русского на английский
df = df.rename(columns={'Процессор':'cpu', 'Оперативная память': 'ram', 'Видеопроцессор': 'graphics', 'Жесткий диск': 'storage', 'Операционная система': 'operating_system', 'Страна производства': 'country', 'Гарантийный срок': 'warranty'})

In [240]:
# удаление колонок, не представляющих ценности для анализа
columns = ['Габариты товара', 'Габариты товара (с упаковкой)', 'seller']
df.drop(columns=columns, inplace=True)

In [241]:
# проверка списка колонок датафрейма
df.columns

Index(['product_id', 'title', 'price', 'sales', 'feedbacks', 'seller_rating',
       'cpu', 'ram', 'storage', 'graphics', 'operating_system', 'warranty',
       'country'],
      dtype='object')

In [242]:
# провервка на дубликаты
df.duplicated().sum()

490

In [243]:
# убираем дубликаты
df.drop_duplicates(inplace=True)

In [244]:
# проверка 4500 - 490 = 4010, дублирующие колонки удалены
df.shape

(4010, 13)

In [245]:
# перезаписываем индексы
df.reset_index(drop=True, inplace=True)

## **Проверка пропущенных значений**

In [246]:
df.isna().sum()

product_id             0
title                  0
price                  1
sales               2975
feedbacks              0
seller_rating         97
cpu                    0
ram                    0
storage                0
graphics               0
operating_system       0
warranty            1644
country             1686
dtype: int64

## **Несколько колонок имеют пропущенные значения:**
- sales - 2975, более половины датасета, исключение такого количества данных может привести к ошибочным выводам, оставим их, есть вероятность, что по этим товарам не было продаж
- seller_rating, seller - удалим пустые значения или колонки.
- warranty - 1644, проверим.
- country - 1686, проверим какие страны представлены.

## ***title***

In [247]:
# приведение названий к нижнему регистру
df ['title'] = df['title'].str.lower()

In [248]:
# проверка
df['title'].value_counts()

компьютер, системный блок, пк intel i5                    33
компьютер rgb, системный блок, пк intel i7                33
компьютер, системный блок, пк intel i7                    33
компьютер rgb, системный блок, пк intel i5                33
компьютер rgb, системный блок, пк intel i3                33
                                                          ..
мощный пк robotcomp анаконда 2.0 v2 plus                   1
игровой компьютер robotcomp robotcomp м16 3.0 v3 plus      1
мощный компьютер robotcomp robotcomp м16 3.0 v3 plus       1
мощный компьютер robotcomp robotcomp м16 3.0 v2 plus       1
игровой компьютер robotcomp аллигатор 2.0 bluetooth v1     1
Name: title, Length: 2007, dtype: int64

In [249]:
# так как, заказчик планирует продавать только ПК, то отфильтруем title по содержанию слов 'компьютер' и 'пк'
df = df[df.title.str.contains('компьютер', 'пк')]
df.title.unique()

array(['компьютер hp prodesk 400 g6 mt',
       'офисный компьютер robotcomp казначей v1',
       'игровой компьютер robotcomp м16 2.0 v1', ...,
       'офисный компьютер life tech office i5-2400 ram 8gb ssd 480gb',
       'компьютер /8gb/ssd-128/hdd-500/монитор 20"',
       ' игровой компьютер '], dtype=object)

In [250]:
# проверка количества оставшихся значений
df.shape

(3413, 13)

Изучив данные о названиях, нашла те, которые не относятся к компьютерам:
мышка, микрокомпьютер, мини, кабель. Удалю все названия, которые содержат эти слова.

In [251]:
# проверка наличия слов "мини-пк" и "моноблок", "мышка", "микрокомпьютер", "кабель"
# 'мини', 'моноблок', 'мышка', 'микрокомпьютер', 'кабель'
ind_mini = df[df.title.str.contains('мини')].index
ind_mono = df[df.title.str.contains('моноблок')].index
ind_mouse = df[df.title.str.contains('мышка')].index
ind_micro = df[df.title.str.contains('микрокомпьютер')].index
ind_cabel = df[df.title.str.contains('кабель')].index

In [252]:
# indexes_to_drop
print(ind_mini)
print(ind_mouse)
print(ind_micro)
print(ind_cabel)
print(ind_mono)

Int64Index([  70,  106,  310,  314,  315,  321,  322,  323,  330,  331,  332,
             339,  340,  341,  357,  358,  359,  363,  364,  365,  366,  367,
             368,  384,  385,  386,  448,  479,  492,  493,  494,  495,  496,
             497,  961, 1151, 1152, 1269, 1377, 1704, 2134, 2811, 2812, 2813,
            2814, 2815, 2816, 2959],
           dtype='int64')
Int64Index([1687], dtype='int64')
Int64Index([713, 994, 1174, 1592, 2204, 2207, 2288, 2297, 3062, 3271, 3994], dtype='int64')
Int64Index([156, 214, 395, 565, 566, 631, 1458], dtype='int64')
Int64Index([], dtype='int64')


Так как по условию нам необходимы только ПК, исключим остальные товары. Также проверерим названия на наличие в них слов "моноблок" или "мини-пк"

In [253]:
df.drop(axis=0, index=ind_mini, inplace=True)

<ipython-input-253-34d0d6f8ac78>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(axis=0, index=ind_mini, inplace=True)


In [254]:
df.drop(axis=0, index=ind_mouse, inplace=True)

<ipython-input-254-ec0f3da5c2f7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(axis=0, index=ind_mouse, inplace=True)


In [255]:
df.drop(axis=0, index=ind_micro, inplace=True)

<ipython-input-255-faf421cda789>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(axis=0, index=ind_micro, inplace=True)


In [256]:
df.drop(axis=0, index=ind_cabel, inplace=True)

<ipython-input-256-fce4c03e0077>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(axis=0, index=ind_cabel, inplace=True)


In [257]:
ind_mini_2 = df[df.title.str.contains('mini')].index
ind_mini_2

Int64Index([190, 192, 193, 197, 269, 270, 1777, 1778, 1779, 1780, 1781, 1782], dtype='int64')

In [258]:
df.drop(axis=0, index=ind_mini_2, inplace=True)

In [259]:
# проверка количества оставшихся колонок
df.shape

(3334, 13)

In [260]:
# проверка значений
df.title.value_counts()

компьютер rgb, системный блок, пк intel i5                                    33
компьютер, системный блок, пк intel i5                                        33
компьютер, системный блок, пк intel i3                                        33
компьютер, системный блок, пк intel i7                                        33
компьютер rgb, системный блок, пк intel i7                                    33
                                                                              ..
игровой компьютер roo24 i3 valentine                                           1
игровой компьютер roo24 i3 morpheus                                            1
компьютер для учебы и игр 8gb/ssd-256+монитор-20"                              1
компьютер для игр и учебы - i7/gtx-650/8gb/ssd-128gb/hdd-500gb/монитор-22'     1
компьютер thinkcentre m70qgen2tiny ci7-11700t/16/512gb                         1
Name: title, Length: 1479, dtype: int64

## ***price***

In [261]:
# проверим количество пустых значений
df.price.isnull().sum()

0

In [262]:
# проверим, что из себя представляют данные
df.price

2        35720₽
3        39237₽
4        76188₽
5        55625₽
6       128284₽
         ...   
4000     77250₽
4001     77250₽
4002     77250₽
4003     77250₽
4004     77250₽
Name: price, Length: 3334, dtype: object

Очистим от символа ₽, преобразуем в числовой формат:

In [263]:
# функция для очистки от нечисловых значений и перевода в числовой формат
def clear_data(data):
  return int(re.sub('\D', '', data))

In [264]:
# применение функции
df['price'] = df['price'].apply(clear_data)

In [265]:
# проверка значений
df.price.sort_values()

85         254
581        600
79         605
78         662
580        700
         ...  
1098    209955
1101    214954
1100    214954
3598    215900
3615    235900
Name: price, Length: 3334, dtype: int64

In [266]:
# Видим, что значения цены начинается от 254 руб, что кажется нереальной, поэтому проверим все товары со стоимостью ниже 15000 руб
df[df['price'] < 15000]

,product_id,title,price,sales,feedbacks,seller_rating,cpu,ram,storage,graphics,operating_system,warranty,country
77,37982087,компьютер,777,Купили менее 5 раз,0 отзывов,4.7,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
78,37982255,компьютер,662,Купили менее 5 раз,0 отзывов,4.7,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
79,37982823,компьютер,605,Купили более 5 раз,0 отзывов,4.7,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
80,38113745,компьютер,1065,Купили менее 5 раз,1 отзыв,4.7,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
81,38114033,компьютер,892,NaN,0 отзывов,4.7,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
85,38491468,компьютер,254,Купили более 40 раз,2 отзыва,4.6,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
87,38721001,компьютер,4789,NaN,0 отзывов,4.4,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
88,38721347,компьютер,3355,Купили менее 5 раз,0 отзывов,4.4,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,Китай
96,43110911,компьютер,10965,NaN,0 отзывов,3.4,"{'Процессор_тип': 'не заполнено', 'Количество ...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,отсутствует,NaN,США
155,50004971,компьютер iru home 120,11540,Купили менее 5 раз,1 отзыв,4.4,"{'Процессор_тип': '6010', 'Количество ядер про...","{'Тип оперативной памяти': 'не заполнено', 'Об...","{'Объем накопителя HDD': None, 'Объем накопите...",не заполнено,Free DOS,3 года,Россия


Видим большое количество пропущенных значений для товаров, стоимостью ниже 10000 руб.
На основе нереалестичной цены и пропущенных значений делаем вывод, что данные не являются ценными для анализа, поэтому удаляем их.

In [267]:
# df = df[df.title.str.contains('компьютер', 'пк')]
df = df[df['price'] > 10000]

In [268]:
# проверка количества оставшихся значений
df.shape

(3319, 13)

## ***saller_rating***

In [269]:
# просмотр значений
df['seller_rating'].sort_values()

4004    0.0
3390    0.0
3389    0.0
3388    0.0
3383    0.0
       ... 
1691    NaN
1692    NaN
2787    NaN
2875    NaN
2876    NaN
Name: seller_rating, Length: 3319, dtype: float64

In [270]:
# проверка пропущенных значений
df.seller_rating.isna().sum()

85

In [271]:
# удаление пропущенных значений
df.dropna(subset=['seller_rating'], inplace=True)

In [272]:
# проверка количества оставшихся значений
df.shape

(3234, 13)

## ***sales***

In [273]:
# посмотрим, что представляют из себя данные
df.sales.unique()

array(['Купили более 10 раз', 'Купили более 200 раз',
       'Купили более 40 раз', 'Купили более 5 раз', 'Купили более 30 раз',
       'Купили более 20 раз', 'Купили более 50 раз', 'Купили менее 5 раз',
       'Купили более 400 раз', 'Купили более 500 раз',
       'Купили более 100 раз', 'Купили более 80 раз', nan,
       'Купили более 90 раз', 'Купили более 300 раз',
       'Купили более 70 раз', 'Купили более 60 раз',
       'Купили более 800 раз'], dtype=object)

Видим, что есть пропущенные значения, данные содержат числовые и буквенные символы, также видим, что данные содержат не конкретное число продаж, а интервал, в котором расположено число продаж, то есть они сгруппированны по количеству продаж.В этом случае не имеет смысл очищать данные от нечиловых символов, а лучше расположить группы в порядке увеличения продаж и задать им ранги. Приведем количество продаж к единым рангам, кратным 100.
Примем допущение, что ПК, для которых отсутствуют данные о продажах, не были проданы ни разу, заполним пустые значения 0.
Сформируем список индексов, значения продаж которых равны 0

In [274]:
# задаем словарь для ранжирования
dct = {'Купили менее 5 раз': 1,
       'Купили более 5 раз': 1,
       'Купили более 10 раз': 1,
       'Купили более 20 раз': 1,
       'Купили более 30 раз': 1,
       'Купили более 40 раз': 1,
       'Купили более 50 раз': 1,
       'Купили более 60 раз': 1,
       'Купили более 70 раз': 1,
       'Купили более 80 раз': 1,
       'Купили более 90 раз': 1,
       'Купили более 100 раз': 2,
       'Купили более 200 раз': 3,
       'Купили более 300 раз': 4,
       'Купили более 400 раз': 5,
       'Купили более 500 раз': 6,
       'Купили более 800 раз': 7
       }

In [275]:
# применяем ранжирование
df['sales_range'] = df['sales'].map(dct)

In [276]:
# проверяем результат
df['sales_range'].value_counts()

1.0    712
2.0     16
3.0      5
5.0      2
6.0      2
4.0      2
7.0      1
Name: sales_range, dtype: int64

In [277]:
# заполняем пустые значения 0
df.sales_range = df.sales_range.fillna(0)

In [278]:
# проверка результата
df['sales_range'].value_counts()

0.0    2494
1.0     712
2.0      16
3.0       5
5.0       2
6.0       2
4.0       2
7.0       1
Name: sales_range, dtype: int64

In [279]:
# список индексов, значения продаж которых равны 0
index_sales = df[df['sales_range']==0].index
index_sales

Int64Index([  72,   74,   96,  107,  109,  118,  121,  122,  188,  194,
            ...
            3993, 3995, 3997, 3998, 3999, 4000, 4001, 4002, 4003, 4004],
           dtype='int64', length=2494)

У нас есть пропущенные значения в  sales, но в feedback для этих товаров есть информация о количестве отзывов. Так как отзыв может быть оставлен только на купленный товар, то значение sales_range будет больше или равно значению feedback.

## ***feedbacks***

In [280]:
# посмотрим, что представляют из себя данные
df.feedbacks.unique()

array([' 1 отзыв', ' 94 отзыва', ' 12 отзывов', ' 4 отзыва', ' 9 отзывов',
       ' 3 отзыва', ' 2 отзыва', ' 6 отзывов', ' 17 отзывов',
       ' 25 отзывов', ' 8 отзывов', ' 5 отзывов', ' 0 отзывов',
       ' 101 отзыв', ' 128 отзывов', ' 54 отзыва', ' 42 отзыва',
       ' 84 отзыва', ' 7 отзывов', ' 21 отзыв', ' 14 отзывов',
       ' 16 отзывов', ' 38 отзывов', ' 13 отзывов', ' 148 отзывов',
       ' 30 отзывов', ' 18 отзывов', ' 47 отзывов', ' 34 отзыва',
       ' 89 отзывов', ' 19 отзывов', ' 26 отзывов', ' 66 отзывов',
       ' 11 отзывов', ' 39 отзывов', ' 15 отзывов', ' 296 отзывов',
       ' 22 отзыва', ' 189 отзывов', ' 74 отзыва', ' 10 отзывов',
       ' 117 отзывов', ' 78 отзывов', ' 59 отзывов', ' 27 отзывов',
       ' 29 отзывов', ' 80 отзывов'], dtype=object)

Видим, что данные представляют собой тип object, который содержит число и слово "отзыв". Очисим данные от нечисловых символов и приведем к приведем к целочисленному типу (int).
Пустые значения отсутсвуют.

In [281]:
# применение функции
df['feedbacks'] = df['feedbacks'].apply(clear_data)

In [282]:
# оцениваем полученный результат
df.feedbacks.unique()

array([  1,  94,  12,   4,   9,   3,   2,   6,  17,  25,   8,   5,   0,
       101, 128,  54,  42,  84,   7,  21,  14,  16,  38,  13, 148,  30,
        18,  47,  34,  89,  19,  26,  66,  11,  39,  15, 296,  22, 189,
        74,  10, 117,  78,  59,  27,  29,  80])

In [283]:
# список индексов, значения отзывов которых  не равны 0
ind_feedbacks = df[df['feedbacks'] != 0].index
ind_feedbacks

Int64Index([   3,    4,    5,    6,    7,    8,    9,   10,   11,   12,
            ...
            3074, 3075, 3557, 3715, 3748, 3774, 3775, 3776, 3777, 3985],
           dtype='int64', length=571)

In [284]:
# проверим, есть ли такие индексы, для которых есть данные о количестве отзывов, но нет данных о продажах и  сколько их
count = 0
ind_list = []
for ind in ind_feedbacks:
  for index in index_sales:
    if index == ind:
      ind_list.append(index)
      count += 1
print(ind_list)
print(f'Количество индексов, для которых есть данные о количествах отзывов, но нет данных о продажах: {count}')

[836, 960, 982, 987, 1105, 1107, 1116, 1120, 1205, 1279, 1294, 1363, 1393, 1431, 1443, 1594, 1611, 1612, 1613, 1617, 1626, 1670, 1697, 1709, 1713, 1719, 1739, 1861, 1866, 1874, 1877, 1879, 1880, 1881, 1890, 1907, 2152, 2153, 2163, 2169, 2173, 2174, 2180, 2184, 2185, 2186, 2468, 2469, 2471, 2472, 2473, 2474, 2475, 2477, 2478, 2681, 2682, 2683, 2684, 2685, 2686, 2688, 2689, 2690, 2691, 2786, 2788, 2789, 2790, 2794, 2796, 2797, 2798, 2800, 2803, 2949, 3074, 3075, 3557, 3715, 3748, 3774, 3775, 3776, 3777, 3985]
Количество индексов, для которых есть данные о количествах отзывов, но нет данных о продажах: 86


Индексов, для которых есть данные об отзывах, но нет данных о продажах - 86.
Исходя из того, что только те, кто приобрел товар, может оценить его и оставить отзыв, число продаж должно быть больше или равно отзывам. Для строк с найденными индексами внесем данные о подажах, равные данным о количестве отзывов.

In [285]:
# заполним данные о подажах для тех значений, для которых есть отзывы, но не указаны продажи, сравнивая количество отзывов с разбивкой по рангам для sales_range
for ind in ind_list:
  if df['feedbacks'][ind] <= 100:
    df['sales_range'][ind] = 1
  elif df['feedbacks'][ind] <= 200:
    df['sales_range'][ind] = 2


<ipython-input-285-4448f925a904>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sales_range'][ind] = 1
<ipython-input-285-4448f925a904>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sales_range'][ind] = 2


In [286]:
# проверка  количесва значений = 0
df[df['sales_range'] == 0].shape

(2408, 14)

In [287]:
# проверка корректности записи значений
df.sales_range.unique()

array([1., 3., 5., 6., 2., 0., 4., 7.])

## ***cpu***

In [288]:
# проверим на наличие пустых значений:
df.cpu.isna().sum()

0

In [289]:
# посмотрим, что из себя представляют данные в колонке
df.cpu.unique()

array(["{'Процессор_тип': 'Intel Core i5', 'Количество ядер процессора': '6'}",
       "{'Процессор_тип': 'Intel Core i5', 'Количество ядер процессора': '10'}",
       "{'Процессор_тип': 'Intel Core i3', 'Количество ядер процессора': '4'}",
       "{'Процессор_тип': 'AMD Ryzen 5', 'Количество ядер процессора': '6'}",
       "{'Процессор_тип': 'AMD Athlon', 'Количество ядер процессора': '2'}",
       "{'Процессор_тип': 'Intel Core i7', 'Количество ядер процессора': '12'}",
       "{'Процессор_тип': 'Intel Core i9', 'Количество ядер процессора': '16'}",
       "{'Процессор_тип': 'AMD Ryzen 3', 'Количество ядер процессора': '2'}",
       "{'Процессор_тип': 'Gemini Lake Refresh J4115', 'Количество ядер процессора': '4'}",
       "{'Процессор_тип': 'Intel Core 12400F', 'Количество ядер процессора': '6'}",
       "{'Процессор_тип': 'Intel Core i7', 'Количество ядер процессора': '8'}",
       "{'Процессор_тип': 'не заполнено', 'Количество ядер процессора': 'не заполнено'}",
       "{'Процессо

In [290]:
# это словарь, распакуем его, создадим 2 новых столбца
df['cpu_type'] = df['cpu'].apply(lambda x: eval(x)['Процессор_тип'])

In [291]:
# оцениваем полученный результат
df['cpu_type'].unique()

array(['Intel Core i5', 'Intel Core i3', 'AMD Ryzen 5', 'AMD Athlon',
       'Intel Core i7', 'Intel Core i9', 'AMD Ryzen 3',
       'Gemini Lake Refresh J4115', 'Intel Core 12400F', 'не заполнено',
       '6010', '12400f', 'AMD A6', 'Intel Pentium', '8 ядер',
       'Intel 4 ядра', 'AMD A12-9800E', 'Intel Xeon', 'AMD Ryzen 7',
       'AMD E1-6010', 'AMD RYZEN 4600g Vega7 арт. 100343717',
       'Intel N5105', 'AMD Ryzen 6', 'Intel Celeron',
       'AMD RYZEN 4600g Vega7 арт. 143553804', 'Intеl Quad Q9400',
       'Intеl Quad 9400', 'Intel E5', 'Intel Core Quad 9400', 'Intel E3',
       'AMD Radeon RX 570', 'Q9400', 'Intеl Quad', 'Ryzen; 3 1300x',
       'intel core i 5 10400f', 'AMD рязань 5 5600x', 'AMD рязань 5 4650',
       'AMD Ryzen 9'], dtype=object)

**План по очистке данных:**
1. Приведем все значения к нижнему регистру.
2. Проверим есть ли грамматичские ошибки или описки или одни и те же названия на разных языках, исправим при необходимости.
3. Проверим количество не запалненных значений.

In [292]:
# приведение значений к нижнему регистру
df['cpu_type'] = df['cpu_type'].str.lower()

In [293]:
# проверка
df['cpu_type'].value_counts()

intel core i5                           1011
intel core i7                            652
intel core i3                            445
amd ryzen 5                              432
не заполнено                             229
amd athlon                               118
intel xeon                                72
amd ryzen 7                               62
amd ryzen 3                               46
amd a6                                    45
intel pentium                             30
intel e5                                  22
intel core i9                             13
intеl quad                                11
intel e3                                  11
amd a12-9800e                              4
q9400                                      3
amd ryzen 9                                3
intel core 12400f                          3
amd ryzen 6                                2
intel celeron                              2
amd radeon rx 570                          2
intel 4 яд

- intеl quad 9400 не существует, вместо него intеl quad q9400, заменим, также заменин на него "intel core quad 9400"
- встречается слово "рязань", которое заменим на "ryzen"
- 6010 - с таким номером существует только один процессор, это amd e1-6010, заменим на него
- в двух названиях содержатся артикулы, уберем их
- "intel core 12400f" относится к процессору intel core i5, заменим на это название,
- "12400f" также заменим на "intel core i5"
- удалим те, которые имеют значение "не заполнено"
- по значению "8 ядер" не понятно, к какому типу процессора относится, удалим
- "amd radeon rx 570" это видеокарта, а не процессор, поэтому удалим
- создадим столбец с данными о бренде процессора                    1



In [294]:
# фильтрация датасета и исключение данных, не подлежащих восстановлению
df = df[(df['cpu_type'] != 'не заполнено') & (df['cpu_type'] != 'amd radeon rx 570') & (df['cpu_type'] !='8 ядер')]

In [295]:
# функция для поиска значений и их замены
def cpu_clear(data):
  if re.findall(r'рязань', data):
    return re.sub(r'рязань', 'ryzen', data)
  elif re.findall(r'12400f', data):
    return 'intel core i5'
  elif re.findall(r'арт. ', data):
    return 'amd ryzen 5'
  elif re.findall(r'6010', data):
    return 'amd e1-6010'
  elif re.findall(r'10400f', data):
    return 'intel core i5'
  elif re.findall(r'ryzen 5', data):
    return 'amd ryzen 5'
  elif re.findall(r'n5105', data):
    return 'intel celeron'
  elif re.findall(r'gemini lake refresh j4115', data):
    return 'intel celeron'
  elif re.findall(r'ryzen;', data):
    return 'amd ryzen 3'
  elif re.findall(r'amd ryzen 5 5600x', data):
    return 'amd ryzen 5'
  elif re.findall(r'quad', data):
    return 'intеl quad'
  else:
    return data

In [296]:
# применение функции
df['cpu_t'] = df['cpu_type'].apply(cpu_clear)

<ipython-input-296-47edc05d399b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cpu_t'] = df['cpu_type'].apply(cpu_clear)


In [297]:
# проверка получившихся значений
df['cpu_t'].value_counts()

intel core i5        1016
intel core i7         652
intel core i3         445
amd ryzen 5           434
amd athlon            118
intel xeon             72
amd ryzen 7            62
amd ryzen 3            47
amd a6                 45
intel pentium          30
intel e5               22
intеl quad             14
intel core i9          13
intel e3               11
amd a12-9800e           4
intel celeron           4
q9400                   3
amd ryzen 9             3
amd ryzen 6             2
amd e1-6010             2
amd ryzen 5 5600x       1
amd ryzen 5 4650        1
intel 4 ядра            1
Name: cpu_t, dtype: int64

Оставим значения, количество которых больше 11

In [298]:
# создание датасета с именем процессора и их количеством в основном датасете
df_cpu = df['cpu_t'].value_counts()

In [299]:
# создание списка индексов (имен процессоров)
df_cpu_ind = df_cpu.index

In [300]:
# создание списка имен процессоров, которые представляют ценность для анализа (количество которых больше 11)
df_cpu_important = df_cpu[df_cpu > 10]

In [301]:
df_cpu_important

intel core i5    1016
intel core i7     652
intel core i3     445
amd ryzen 5       434
amd athlon        118
intel xeon         72
amd ryzen 7        62
amd ryzen 3        47
amd a6             45
intel pentium      30
intel e5           22
intеl quad         14
intel core i9      13
intel e3           11
Name: cpu_t, dtype: int64

In [302]:
# фильтрация датасета по именам процессоров, представляющим ценность для анализа
df = df[df['cpu_t'].isin(df_cpu_important.index)]

In [303]:
# проверка количества оставшихся строк
df.shape

(2981, 16)

Создадим еще одну колонку, содержащую название бренда процессора и далее проверим, является ли этот признак важным для ценообразования

In [304]:
# функция для создания колонки с именем бренда
def cpu_brand(data):
  if re.findall(r'amd', data):
    return 'amd'
  elif re.findall(r'ryzen', data):
    return 'amd'
  elif re.findall(r'intel', data):
    return 'intel'
  elif re.findall(r'intеl', data):
    return 'intel'
  else:
    return data

In [305]:
# применение функции
df['cpu_brand'] = df['cpu_t'].apply(cpu_brand)

<ipython-input-305-c7272dfad710>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cpu_brand'] = df['cpu_t'].apply(cpu_brand)


In [306]:
# проверка значений
df['cpu_brand'].value_counts()

intel    2275
amd       706
Name: cpu_brand, dtype: int64

Формируем столбец с количеством ядер процессора

In [307]:
# распаковывая словарь
df['cores'] = df['cpu'].apply(lambda x: eval(x)['Количество ядер процессора'])

In [308]:
# оцениваем полученный результат
df['cores'].unique()

array(['6', '10', '4', '2', '12', '16', '8', '4 ядра', '8 ядер',
       'не заполнено', '14', '20', '24 ядра'], dtype=object)

Данные о количестве ядер представлены как в виде числовых значений, так и в виде численно-буквенных, очистим их от нечисловых значений.

In [309]:
# функция для очистки от нечисловых символов
def cores_clear(data):
  return re.sub('\D', '', data)

In [310]:
# применяем функцию
df['cores'] = df['cores'].apply(cores_clear)

In [311]:
df['cores'] = pd.to_numeric(df['cores'])

In [312]:
df['cores'].unique()

array([ 6., 10.,  4.,  2., 12., 16.,  8., nan, 14., 20., 24.])

In [313]:
# проверкаа пропущенных значений
df.cores.isna().sum()

1

In [314]:
# удаление пропущенных значений
df.dropna(subset='cores', inplace=True)

## ***ram***

In [315]:
# посмотрим, что из себя представляют данные в колонке
df.ram.unique()

array(["{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '8'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '16'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '16 ГБ'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '32'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '32 ГБ'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '8 ГБ'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '16 гб'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '4'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '64'}",
       "{'Тип оперативной памяти': 'DDR 4', 'Объем оперативной памяти (Гб)': '240 гб'}",
       "{'Тип оперативной памяти': 'DDR 3', 'Объем оперативной памяти (Гб)': '8 ГБ'}",
       "{'Тип оперативной памяти': 'DDR 3', 'Объем

In [316]:
# распаковываем словарь, формируем новый столбец
df['ram_type'] = df['ram'].apply(lambda x: eval(x)['Тип оперативной памяти'])

In [317]:
# распаковываем словарь, формируем новый столбец
df['ram_size'] = df['ram'].apply(lambda x: eval(x)['Объем оперативной памяти (Гб)'])

In [318]:
# посмотрим результат:
df['ram_type'].value_counts()

DDR 4           2671
DDR 3            246
не заполнено      56
DDR 5              4
16                 2
16 Гб              1
Name: ram_type, dtype: int64

In [319]:
#есть несколько значений, в которых вместо типа указан размер, проверим их
df[df['ram_type'].str.contains('16')]

,product_id,title,price,sales,feedbacks,seller_rating,cpu,ram,storage,graphics,operating_system,warranty,country,sales_range,cpu_type,cpu_t,cpu_brand,cores,ram_type,ram_size
2950,147211674,"игровой компьютер /16gb/ssd-512/монитор-24""",68800,NaN,0,4.5,"{'Процессор_тип': 'Intel E5', 'Количество ядер...","{'Тип оперативной памяти': '16', 'Объем операт...","{'Объем накопителя HDD': None, 'Объем накопите...",NVIDIA GeForce RTX 1060,Windows 11 Pro,1 год,Россия,0.0,intel e5,intel e5,intel,12.0,16,16 ГБ
2952,147213987,"игровой компьютер /16gb/ssd-512/монитор-24""",69120,NaN,0,4.5,"{'Процессор_тип': 'Intel Xeon', 'Количество яд...","{'Тип оперативной памяти': '16 Гб', 'Объем опе...","{'Объем накопителя HDD': None, 'Объем накопите...",NVIDIA GeForce RTX 1060,Windows 11 Pro,1 год,Россия,0.0,intel xeon,intel xeon,intel,12.0,16 Гб,16 ГБ
3266,150787428,игровой компьютер,51300,NaN,0,5.0,"{'Процессор_тип': 'AMD Ryzen 5', 'Количество я...","{'Тип оперативной памяти': '16', 'Объем операт...","{'Объем накопителя HDD': None, 'Объем накопите...",AMD Radeon Vega 7,Windows 10,1 год,Россия,0.0,amd ryzen 5,amd ryzen 5,amd,6.0,16,16 гб


In [320]:
# видим, что для этих продуктов информация о размере оперативной памяти есть, но отсутствут ее тип, поэтому заменим данные на "не заполнено"
df['ram_type'] = df.ram_type.replace('16', 'не заполнено').replace('16 Гб', 'не заполнено')

In [321]:
# проверка успешеносит замены:
df.ram_type.value_counts()

DDR 4           2671
DDR 3            246
не заполнено      59
DDR 5              4
Name: ram_type, dtype: int64

In [322]:
# фильтрация и исключение из датасета значений, которые не заполнены
df = df[df['ram_type']!='не заполнено']

In [323]:
# проверка количества оставшихся строк
df.shape

(2921, 20)

In [324]:
# проверим, нужна ли очистка для показателя 'ram_size'
print(df['ram_size'].value_counts())

16 ГБ      1273
32 ГБ       405
16          384
8 ГБ        264
32          218
8           125
4 ГБ         69
16 Gb        60
16 гб        47
64           29
32 гб        15
16 gb        14
4             8
8 гб          5
64 ГБ         2
1000 гб       1
64 гб         1
240 гб        1
Name: ram_size, dtype: int64


In [325]:
# очистим данные в колонке ram_size от букв и переведем в чиловой формат, для этого воспользуемся функцией с регулярными выражениями
def ram_clear(data):
  if pd.isna(data):
    return np.nan
  else:
    return re.sub('\D', '', data)

In [326]:
# применяем функцию
df['ram_size'] = df['ram_size'].apply(ram_clear)

In [327]:
# переводим данные в числовой формат
df['ram_size'] = df.ram_size.astype('Int64')

In [328]:
# проверка значений после очистки
df.ram_size.unique()

<IntegerArray>
[8, 16, 32, 4, 64, 240, 1000]
Length: 7, dtype: Int64

In [329]:
# проверим на отсутствие значений
df.ram_size.isna().sum()

0

In [330]:
# уберем аномальные значения
df = df[df['ram_size']<240]

In [331]:
# проверка количества оставшихся строк
df.shape

(2919, 20)

## ***storage***

Распакуем словари, создадим 4 колонки:
- объем ssd
- объем hhd
- диск ssd
- диск hhd
Затем очистим данные в этих колонках, затем объединим данные из колонок объем ssd и объем hhd в storage_value и диск ssd и диск hhd в storage_type.

In [332]:
# посмотрим что из себя представляют данные колонки
df['storage']

3       {'Объем накопителя HDD': None, 'Объем накопите...
4       {'Объем накопителя HDD': None, 'Объем накопите...
5       {'Объем накопителя HDD': None, 'Объем накопите...
6       {'Объем накопителя HDD': None, 'Объем накопите...
7       {'Объем накопителя HDD': None, 'Объем накопите...
                              ...                        
4000    {'Объем накопителя HDD': '1000 Гб', 'Объем нак...
4001    {'Объем накопителя HDD': '1000 Гб', 'Объем нак...
4002    {'Объем накопителя HDD': '1000 Гб', 'Объем нак...
4003    {'Объем накопителя HDD': '1000 Гб', 'Объем нак...
4004    {'Объем накопителя HDD': '1000 Гб', 'Объем нак...
Name: storage, Length: 2919, dtype: object

In [333]:
# распаковываем словарь
# df['cpu_type'] = df['cpu'].apply(lambda x: eval(x)['Процессор_тип'])
df['hhd_value'] = df['storage'].apply(lambda x: eval(x)['Объем накопителя HDD'])

In [334]:
# Создаем новую колонку, содержащую данные о SSD
df['ssd_value'] = df['storage'].apply(lambda x: eval(x)['Объем накопителя SSD'])

В данных видим несколько проблем:
- присутствие номеров арт., которые помешают использованию регулярных выражений
- присутствие букв, которые убрем при помощи регулярных выражений
- есть данные в Гб и Тб, после преобразования в числовое значение, умножим на 1000 однозначные

In [335]:
# проверка значений
df.hhd_value.unique()

array([None, '1000 Гб', '2000 Гб', '1 Тб', '1 тб', '2000Gb', '1 TB',
       'нет', 'нет; без HDD', 'без HDD', '500 Gb', '1000 Gb', '500',
       '3000 гб', '500 Гб', '2 ТБ', '500 гб', '512 Гб', 'Отсутствует',
       'отсутствует', '1000', '250 гб', '4000 Гб; 4000', '1000 гб',
       '1024 ГБ', '2 тб'], dtype=object)

In [336]:
# функция для очистки нечисловых значений
def clear_hhd(data):
  if pd.isna(data):
    return np.nan
  elif re.findall(r'\D+\d+', data):
    return re.sub(r'\D+\d+', '', data)
  else:
    return re.sub(r'\D', '', data)

In [337]:
# применение функции для очистки от нечисловых значений
df.hhd_value = df.hhd_value.apply(clear_hhd)

In [338]:
# функция для замены близких значений (оставим 120, 250, 500, 1000)
def storage_val(data):
  if pd.isna(data):
    return np.nan
  elif re.findall(r'128', data):
    return re.sub(r'128', '120', data)
  elif re.findall(r'256', data):
    return re.sub(r'256', '250', data)
  elif re.findall(r'240', data):
    return re.sub(r'240', '250', data)
  elif re.findall(r'480', data):
    return re.sub(r'480', '500', data)
  elif re.findall(r'512', data):
    return re.sub(r'512', '500', data)
  elif re.findall(r'960', data):
    return re.sub(r'960', '1000', data)
  elif re.findall(r'64', data):
    return re.sub(r'64', '60', data )
  elif re.findall(r'1024', data):
    return re.sub(r'1024', '1000', data)
  else:
    return data

In [339]:
# применение функции замены близких значений
df['hhd_value'] = df['hhd_value'].apply(storage_val)

In [340]:
# проверка получившегося результата
df.hhd_value.unique()

array([nan, '1000', '2000', '1', '', '500', '3000', '2', '250', '4000'],
      dtype=object)

In [341]:
# приведение полученных значений к числовому формату
df['hhd_value'] = pd.to_numeric(df['hhd_value'])

In [342]:
# данные были записаны как в Гб, так и Тб, преобразует Тб в Гб. Значения в ТБ представлены в виде однозначных чисел, умножим их н 1000
def to_gb(data):
  if pd.isna(data):
    return np.nan
  elif data < 5.0:
    return data * 1000
  else:
    return data

In [343]:
# применение функции преобразования Тб в Гб
df['hhd_value'] = df['hhd_value'].apply(to_gb)

In [344]:
# заполнение пропущенных значений 0
df['hhd_value'] = df['hhd_value'].fillna(0)

In [345]:
# проверка результата
df['hhd_value'].unique()

array([   0., 1000., 2000.,  500., 3000.,  250., 4000.])

Создадим колонку, в которую внесем информацию, является ли диск hhd

In [346]:
# функция для определения, является ли диск hhd
def storage_type(data):
  if data != 0:
    return 'hhd'
  else:
    return ''

In [347]:
# применение функции
df['hhd_type'] = df['hhd_value'].apply(storage_type)

In [348]:
# проверка результата
df['hhd_type'].unique()

array(['', 'hhd'], dtype=object)

Работаем с колонкой ssd_value

In [349]:
# проверка значений
df.ssd_value.value_counts()

480 ГБ                   758
1000 Гб                  562
512 Гб                   230
960 Гб                   152
240 Гб                   119
256                      119
480Gb                    117
480                       70
256 Гб                    66
240 Gb                    47
120 Gb                    47
1000 гб                   42
512                       41
240Gb                     40
960Gb                     35
240                       32
128                       32
480 GB                    27
480 Gb                    27
120 GB                    27
240 GB                    27
512 гб                    21
1 Тб                      20
512 Gb                    20
500 Гб                    20
2 Тб                      19
120 ГБ                    15
500 гб                    15
1000 GB                   14
500                       13
2000 ГБ                   12
120GB                     10
120                        8
256 гб                     7
480 гб        

In [350]:
# функция для очистки от нечисловых значений
def clear_ssd(data):
  if pd.isna(data):
    return np.nan
  elif re.findall(r'арт', data):
    return re.sub(r'\D+\s\d+', '', data)
  elif re.findall(r'\D+\d+', data):
    return re.sub(r'\D+\d+', '', data)
  else:
    return re.sub('\D', '', data)

In [351]:
# применение функции
df['ssd_value'] = df['ssd_value'].apply(clear_ssd)

In [352]:
# проверка результата
df['ssd_value'].unique()

array(['480', '960', '512', '2', '240', '1000', '120', '256', '128', nan,
       '250', '500', '1240', '1480', '1', '2000', '64', '1500', '256 гб'],
      dtype=object)

Остались нечисловые символы в данных, создадим функцию

In [353]:
# функция для очистки от нечисловых значений
def clear_storage(data):
  if pd.isna(data):
    return np.nan
  else:
    return re.sub(r'\D', '', data)

In [354]:
# применение функции
df['ssd_value'] = df.ssd_value.apply(clear_storage)

In [355]:
# проверка полученных значений
df['ssd_value'].value_counts()

480     1012
1000     623
512      318
240      271
256      193
960      187
120      111
500       48
128       33
1         22
2         20
2000      17
250       10
1240       6
1480       6
1500       4
64         1
Name: ssd_value, dtype: int64

In [356]:
# применение функции замены близких значений, использованной ранее
df['ssd_value'] = df['ssd_value'].apply(storage_val)

In [357]:
# проверка результата
df['ssd_value'].value_counts()

500     1378
1000     810
250      474
120      144
1         22
2         20
2000      17
1500      10
1250       6
60         1
Name: ssd_value, dtype: int64

In [358]:
# перевод данных в числовой формат
df['ssd_value'] = df['ssd_value'].astype('Int64')

In [359]:
# применение функции перевода Тб в Гб
df['ssd_value'] = df['ssd_value'].apply(to_gb)

In [360]:
# проверка наличия пропущенных значений
df['ssd_value'].isna().sum()

37

In [361]:
# замена пропущенных значений на 0
df['ssd_value'] = df['ssd_value'].fillna(0)

In [362]:
# проверяем данные, однозначных данных не осталось, но есть пропущенные значения, оставим их, так как не все ПК имеют HDD, часть содержит SSD
df['ssd_value'].unique()

array([ 500., 1000., 2000.,  250.,  120.,    0., 1250., 1500.,   60.])

In [363]:
# создание столбца, в котором складываем значение объемов дисков ssd и hhd
df['stor_value'] = df['ssd_value'] + df['hhd_value']

In [364]:
# проверка результата
df['stor_value'].unique()

array([ 500., 1000., 1500., 2000.,  250., 2500., 1250., 1120.,  120.,
        620.,  750., 3500., 3000.,   60., 4000., 6000., 2120.,    0.])

Создадим колонку, в которую внесем информацию, является ли диск ssd

In [365]:
# создание функции для определения, является ли диск ssd
def storage_type_2(data):
  if data != 0:
    return 'ssd'
  else:
    return ''

In [366]:
# применение функции
df['ssd_type'] = df['ssd_value'].apply(storage_type_2)

In [367]:
# проверка результата
df['ssd_type'].unique()

array(['ssd', ''], dtype=object)

In [368]:
# создание колонки, показывающей, имеет ли компьютер диск ssd, hhd или оба
df['stor_type'] =df['hhd_type'] + df['ssd_type']
df['stor_type'].unique()

array(['ssd', 'hhdssd', 'hhd', ''], dtype=object)

In [369]:
# фильтрация датасета по наличию даннных о типе диска
df = df[df['stor_type'] != '']

In [370]:
# проверка результата
df.shape

(2918, 26)

## ***graphics***

In [371]:
# смотрим на, что из себя представляют данные
df['graphics'].unique()

array(['Intel UHD Graphics 630', 'NVIDIA GeForce GTX 1660',
       'NVIDIA GeForce GTX 1650', 'NVIDIA GeForce RTX 3070',
       'NVIDIA GeForce RTX 3050', 'AMD Radeon Vega 7',
       'NVIDIA GeForce RTX 3080', 'AMD Radeon Vega 3',
       'NVIDIA GeForce RTX 3060', 'NVIDIA GeForce GT 1030',
       'intel UHD Graphics 750', 'Intel HD Graphics',
       'NVIDIA GeForce GTX 1050', 'NVIDIA GeForce RTX 2060',
       'AMD Radeon R5', 'Intel UHD Graphics 610',
       'NVIDIA GeForce GTX 1630', 'NVIDIA GeForce GT 730',
       'AMD Radeon RX 6500', 'Intel HD Graphics 2000',
       'Intel HD Graphics 4000', 'не заполнено', 'AMD Radeon Vega 8',
       'AMD Radeon RX 6600', 'Intel HD Graphics 2500',
       'NVIDIA GeForce RTX 4080', 'AMD Radeon RX 550',
       'AMD Radeon RX 470', 'RX 460', 'AMD Radeon RX 580',
       'AMD Radeon Vega 6', 'Radeon R7 350', 'NVIDIA GeForce RTX 1660',
       'NVIDIA GeForce GTX 650', 'NVIDIA GeForce RTX 4070', 'NVIDIA',
       'nvidia', 'Nvidia', 'invidia', 'UHD-график

План действий по очистке:
1. Привести все значения к нижнему регистру.
2. Проверить наличие похожих названий, орфографических ошибок, исправление при необходимости.


In [372]:
# приведение к нижнему регистру и проверка результата
df['graphics'] = df['graphics'].str.lower()
df['graphics'].unique()

<ipython-input-372-cb6020d90999>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['graphics'] = df['graphics'].str.lower()


array(['intel uhd graphics 630', 'nvidia geforce gtx 1660',
       'nvidia geforce gtx 1650', 'nvidia geforce rtx 3070',
       'nvidia geforce rtx 3050', 'amd radeon vega 7',
       'nvidia geforce rtx 3080', 'amd radeon vega 3',
       'nvidia geforce rtx 3060', 'nvidia geforce gt 1030',
       'intel uhd graphics 750', 'intel hd graphics',
       'nvidia geforce gtx 1050', 'nvidia geforce rtx 2060',
       'amd radeon r5', 'intel uhd graphics 610',
       'nvidia geforce gtx 1630', 'nvidia geforce gt 730',
       'amd radeon rx 6500', 'intel hd graphics 2000',
       'intel hd graphics 4000', 'не заполнено', 'amd radeon vega 8',
       'amd radeon rx 6600', 'intel hd graphics 2500',
       'nvidia geforce rtx 4080', 'amd radeon rx 550',
       'amd radeon rx 470', 'rx 460', 'amd radeon rx 580',
       'amd radeon vega 6', 'radeon r7 350', 'nvidia geforce rtx 1660',
       'nvidia geforce gtx 650', 'nvidia geforce rtx 4070', 'nvidia',
       'invidia', 'uhd-графика intel 630', 'intel

Создадим колонку с именем графической карты, проверим, относится ли видеокарта к одному из списка: nvidia, amd, intel

In [373]:
# функция для определения к какому бренду относится видеокарта
def graph_range(data):
  if re.findall(r'nvidia', data):
    return 'nvidia'
  elif re.findall(r'intel', data):
    return 'intel'
  elif re.findall(r'amd', data):
    return 'amd'
  elif re.findall(r'rtx', data):
    return 'nvidia'
  elif re.findall(r'rx', data):
    return 'amd'
  elif re.findall(r'geforce', data):
    return 'nvidia'
  elif re.findall(r'radeon', data):
    return 'amd'
  elif re.findall(r'vega', data):
    return 'amd'
  else:
    return data

In [374]:
# применение функции
df['graphics_range'] = df['graphics'].apply(graph_range)

<ipython-input-374-bc97413d0684>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['graphics_range'] = df['graphics'].apply(graph_range)


In [375]:
# проверка результата
df['graphics_range'].value_counts()

nvidia          2188
amd              419
intel            297
не заполнено      14
Name: graphics_range, dtype: int64

In [376]:
# фильтрация по значениям, которые заполнены и удаление незаполненных
df = df[df['graphics_range'] != 'не заполнено']

In [377]:
# проверка количества оставшихся строк
df.shape

(2904, 27)

## ***operating_system***

In [378]:
# посмотрим в каком виде представлены данные
df['operating_system'].unique()

array(['windows пробная', 'Windows 10 Pro', 'отсутствует',
       'Windows 11 Pro (Trial)', 'Windows 10', 'Windows 7/8/10, LINUX',
       'Windows 10 Home пробная', 'Windows пробная версия',
       'windows 10 trial', 'Windows 10 пробная', 'Windows 11', 'Free DOS',
       'Microsoft Windows 11', 'Window 10 Pro trial', 'Windows', 'DOS',
       'Windows 11 Pro', 'windows 10', 'WINDOWS 10',
       'Windows 10 Pro Trial', 'Windows 10 Home', 'Без OC',
       'Windows 11 Home'], dtype=object)

Видим несколько проблем, которые необходимо исправить:
- приведем все данные к нижнему регистру для удобства
- есть названия одних и тех же операционных систем на русском и английском - приведем к единому виду
- исправим грамматические ошибки или описки

In [379]:
# приводим к нижнему регистру
df['operating_system'] = df['operating_system'].str.lower()
df['operating_system'].value_counts()

<ipython-input-379-39b2cee7f0f4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['operating_system'] = df['operating_system'].str.lower()


windows 10 pro             1579
windows 11 pro              621
windows 10                  352
windows пробная             207
отсутствует                  31
windows 10 pro trial         25
windows 10 пробная           23
windows 11                   19
windows 10 trial             16
windows                      11
windows 10 home               7
windows 10 home пробная       3
windows 11 pro (trial)        2
windows пробная версия        1
free dos                      1
microsoft windows 11          1
window 10 pro trial           1
dos                           1
windows 7/8/10, linux         1
без oc                        1
windows 11 home               1
Name: operating_system, dtype: int64

In [380]:
# функция для очистки данных
def oper_clear(data):
  if re.findall(r'пробная', data) or re.findall(r'trial', data):
    return 'windows пробная'
  elif re.findall(r'10 pro', data):
    return 'windows 10 pro'
  elif re.findall(r'10', data):
    return 'windows 10'
  elif re.findall(r'11 pro', data):
    return 'windows 11 pro'
  elif re.findall(r'11', data):
    return 'windows 11'
  elif re.findall(r'dos', data):
    return 'dos'
  elif re.findall(r'без oc', data):
    return 'отсутствует'
  else:
    return data


In [381]:
# применение функции
df['operating_system'] = df['operating_system'].apply(oper_clear)

<ipython-input-381-e32a7094866a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['operating_system'] = df['operating_system'].apply(oper_clear)


In [382]:
# проверка результата
df['operating_system'].value_counts()

windows 10 pro     1579
windows 11 pro      621
windows 10          360
windows пробная     278
отсутствует          32
windows 11           21
windows              11
dos                   2
Name: operating_system, dtype: int64

## ***warranty***

In [383]:
# смотрим в каком виде представлены данные
df['warranty'].value_counts()

3 Года (36 месяцев)        717
3 года                     457
1 год                      366
12 месяцев                 119
12 месяцев от KING KOMP     23
24 месяца                   21
1 год; 12 мес               12
2 года                       7
1 год; 12 месяцев            7
1 год (12 месяцев)           4
3 месяца                     2
12 месяцев от King Komp      2
6 месяцев                    1
1                            1
14 дней                      1
24 мес                       1
Name: warranty, dtype: int64

План действий:
- приведем данные к нижнему регистру
- очистим данные от нечисловых значений
- приведем все данные к месяцам (есть данные в годах, днях и месяцах)
- переведем данные в числовой тип

In [384]:
# переводим в нижний регистр
df['warranty'] = df['warranty'].str.lower()

<ipython-input-384-403f0b268b6c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['warranty'] = df['warranty'].str.lower()


In [385]:
# функция для очистки, перевода в числовой тип данных и приведения к единой системе измерений (месяц)
def warr_clear(data):
  if pd.isna(data):
    return np.nan
  elif re.findall(r'год', data):                               #найдем данные, которые содержат "год"
    if re.findall(r'\D+\d+\D+', data):                         # для данных, которые содержат в себе количество лет и месяцев (число, текст, число, текст)
      return int(re.sub(r'\D+\d+\D+', '', data)) * 12
    elif re.findall('r\D', data):                              # для данных, которые содержат в себе только год
      return int(re.sub(r'\D', '', data)) *12
  elif re.findall(r'дней', data):                             # для данных, которые содержат дни
    return round(int(re.sub('\D', '', data)) / 30, 1)
  else:
    return int(re.sub('\D', '', data))

In [386]:
# применяем функцию к данным
df['warranty_clean'] = df['warranty'].apply(warr_clear)

In [387]:
# проверка корректности
df['warranty_clean'].value_counts()

36.0    717
12.0    167
24.0     22
3.0       2
6.0       1
1.0       1
0.5       1
Name: warranty_clean, dtype: int64

## ***country***

In [388]:
# смотрим, в каком виде представлены данные
df['country'].unique()

array(['Россия', nan, 'Китай'], dtype=object)

Данные не требуют очистки.

## **Проверка датасета**

In [389]:
# проверка того, как выглядит датасет
df.head(1)

,product_id,title,price,sales,feedbacks,seller_rating,cpu,ram,storage,graphics,...,ram_type,ram_size,hhd_value,ssd_value,hhd_type,stor_value,ssd_type,stor_type,graphics_range,warranty_clean
3,19347937,офисный компьютер robotcomp казначей v1,39237,Купили более 10 раз,1,4.7,"{'Процессор_тип': 'Intel Core i5', 'Количество...","{'Тип оперативной памяти': 'DDR 4', 'Объем опе...","{'Объем накопителя HDD': None, 'Объем накопите...",intel uhd graphics 630,...,DDR 4,8,0.0,500.0,,500.0,ssd,ssd,intel,36.0


In [390]:
# проверка информации о датасете
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2904 entries, 3 to 4004
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        2904 non-null   int64  
 1   title             2904 non-null   object 
 2   price             2904 non-null   int64  
 3   sales             664 non-null    object 
 4   feedbacks         2904 non-null   int64  
 5   seller_rating     2904 non-null   float64
 6   cpu               2904 non-null   object 
 7   ram               2904 non-null   object 
 8   storage           2904 non-null   object 
 9   graphics          2904 non-null   object 
 10  operating_system  2904 non-null   object 
 11  warranty          1741 non-null   object 
 12  country           1389 non-null   object 
 13  sales_range       2904 non-null   float64
 14  cpu_type          2904 non-null   object 
 15  cpu_t             2904 non-null   object 
 16  cpu_brand         2904 non-null   object 


Удаляем колонки: cpu,	ram,	storage, warranty, sales, ssd_type, hhd_type, ssd_value, hhd_value, graphics, cpu_type, перезапишем индексы

In [391]:
columns = ['cpu', 'ram', 'storage', 'warranty', 'sales', 'ssd_type', 'hhd_type', 'ssd_value', 'hhd_value', 'graphics', 'cpu_type', 'country']

In [392]:
df.drop(axis=1, columns=columns, inplace=True)

In [393]:
df = df.reset_index()

In [394]:
df.head(1)

,index,product_id,title,price,feedbacks,seller_rating,operating_system,sales_range,cpu_t,cpu_brand,cores,ram_type,ram_size,stor_value,stor_type,graphics_range,warranty_clean
0,3,19347937,офисный компьютер robotcomp казначей v1,39237,1,4.7,windows пробная,1.0,intel core i5,intel,6.0,DDR 4,8,500.0,ssd,intel,36.0


In [395]:
# удаление колонки index
df.drop(axis=1, columns=['index'], inplace=True)

In [396]:
# сохранение датасета в формате csv
df.to_csv('df_clear.csv', index=False)

In [397]:
# создание датасета, в котором есть данные о продажах
df_sold = df[df['sales_range'] != 0]

In [398]:
# проверка размера датасета
df_sold.shape

(748, 16)

In [399]:
# сохранение датасета в формате csv
df_sold.to_csv('df_sold_clear.csv', index=False)

# **Вывод**

Провела очистку датасета для последующего его анализа.